In [16]:
# Created by Natasha Piccianis, October 12 2022

"""	
Read a fasta file with original transcriptome,
select the longest transcript per each gene and select corresponding lines from GTF file.
Return one FASTA file with nucleotide sequences and one GTF file with selected transcripts.
"""

longestORFperGeneFasta = "/Users/picciani/Dropbox/Yale/YCGA/podocoryna_analysis/reference/isoseq/Cogent_Output/Male_Med.Cogent.hq_transcripts.fasta.no5merge.collapsed.filtered.selected.rep.fa"
transcriptomeFile = "/Users/picciani/Dropbox/Yale/YCGA/podocoryna_analysis/reference/isoseq/Cogent_Output/Male_Med.Cogent.hq_transcripts.fasta.no5merge.collapsed.filtered.rep.fa"
gff = "/Users/picciani/Dropbox/Yale/YCGA/podocoryna_analysis/reference/isoseq/Cogent_Output/Male_Med.Cogent.hq_transcripts.fasta.no5merge.collapsed.filtered.edited.fixed.exons.corrected.gff" # added transcript ID to a new first column; search=(.+transcript_id "(.+)"; gene.+)); replace=$2\t$1
newgtfFile = "/Users/picciani/Dropbox/Yale/YCGA/podocoryna_analysis/reference/isoseq/Cogent_Output/Male_Med.Cogent.hq_transcripts.fasta.no5merge.collapsed.filtered.selected.edited.fixed.exons.corrected.gff"
identifier_type="type_5"

from tkinter.messagebox import YES
import pandas as pd
import re
import argparse
from pathlib import PurePosixPath
from Bio import SeqIO

def getID(header, type, whichID):
	
	"""
	Search the header of a fasta file and return the gene identifier of a sequence."

	Arguments:
	header -- the header string (the record.id value of a sequence when using SeqIO package).
	type -- the type of gene identifier; type_1 = "compXX_cXX_seqXX", type_2="TRINITY_DNXXXXX_cX_gX_iX",
			type_3="SegXX.XX.XXXX".
	whichID -- the kind of identifier to be retrieved; geneID = "gene", transcriptID = "transcript"
	"""

	if type == "type_1":
		searchStr='((comp\d+_c\d+)_seq\d+)'
	if type == "type_2":
		searchStr='((TRINITY_DN\d+_c\d+_g\d+)_i\d+)'
	if type == "type_3":
		searchStr='((Seg\d+)\..+)'
	if type == "type_4":
		searchStr='(t\d+aep.+(DN.+g.+)[_it]\d+)'
	if type == "type_5":
		searchStr = "((PB.\d+).\d+)"
	parts=header.split('\t')
	p=re.match(searchStr, parts[0])
	geneID=p.group(2)
	transcriptID=p.group(1)
		
	if whichID == "gene":
		return geneID
	if whichID == "transcript":
		return transcriptID


# # 	unique = 0 #uncomment for testing
# # 	duplicates = 0 #uncomment for testing
seqs={}
transcriptNames=[]
currentGeneID=''

with open(longestORFperGeneFasta,"w") as outfile1:
	for record in SeqIO.parse(transcriptomeFile,"fasta"): #find gene names
		geneID = getID(record.id, identifier_type, whichID="gene")
		if currentGeneID == '': #start the loop with an empty GeneID string
			currentGeneID = geneID
			seqs[record.description]=[geneID,record.seq] #add first sequence to dictionary "seqs", which has geneID as a list value
		else:
			if geneID != currentGeneID: #if the previous geneID is different from that of the current record
# 					unique += 1 #uncomment for testing
				seqs[record.description]=[geneID,record.seq] #add unique sequences to dictionary "seqs"
				currentGeneID = geneID #re-set current geneID     
			else: #if the geneID is the same as that of the previous record				
# 					duplicates += 1 #uncomment for testing
				for key, value in seqs.items(): #search for the previous record in the dictionary "seqs"
					if value[0] == currentGeneID: #and save its sequence length and name
						dupeLength=(len(value[1]))
						dupeID=key
				if dupeLength<len(record.seq): #if length of previous record is smaller that of than current record
					del seqs[dupeID] #get rid of previous record
					seqs[record.description]=[geneID,record.seq] #add current record to dictionary
					currentGeneID = geneID

	for name, seq in seqs.items(): #print sequences to an output file
		name = name.split(" ")
		name = name[0]
		transcriptNames.append(name)
		print (">" + name, file=outfile1)
		print (seq[1], file=outfile1)

querylist = []
for line in transcriptNames:
	line = line.strip("\n")
	querylist.extend(line.split(" "))

gtf = pd.read_table(gff, header=None)

newgtf = gtf[gtf[0].isin(querylist)] 
newgtf.drop_duplicates(inplace=True)
# #slower code to select the lines
# for name in gtf[0]:  # for transcript name in the first column of gtf file
# 	if (
# 		name not in querylist
# 	):  # if transcript name is not in the list of transcripts to keep, then drop it
# 		index_val = gtf[
# 			gtf[0] == name
# 		].index.values  # get the row number where it is in
# 		gtf.drop(
# 			index=index_val, inplace=True
# 		)  # drop that row and modify the actual file

newgtf.to_csv(
	newgtfFile, sep="\t", quoting=3, header=False, index=False
)  # quoting=3 is the same as quoting=csv.QUOTE_NONE
